# Sentence Reconstruction

The purpose of this project is to take in input a sequence of words corresponding to a random permutation of a given english sentence, and reconstruct the original sentence.

The otuput can be either produced in a single shot, or through an iterative (autoregressive) loop generating a single token at a time.


CONSTRAINTS:
* No pretrained model can be used.
* The neural network models should have less the 20M parameters.
* No postprocessing should be done (e.g. no beamsearch)
* You cannot use additional training data.


BONUS PARAMETERS:

A bonus of 0-2 points will be attributed to incentivate the adoption of models with a low number of parameters.

# Dataset

The dataset is composed by sentences taken from the generics_kb dataset of hugging face. We restricted the vocabolary to the 10K most frequent words, and only took sentences making use of this vocabulary.

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


Download the dataset

In [2]:
from datasets import load_dataset
from keras.layers import TextVectorization
import tensorflow as tf
import numpy as np
np.random.seed(42)
ds = load_dataset('generics_kb',trust_remote_code=True)['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating train split:   0%|          | 0/1020868 [00:00<?, ? examples/s]

Filter row with length greater than 8.


In [3]:
ds = ds.filter(lambda row: len(row["generic_sentence"].split(" "))>8 )
corpus = [ '<start> ' + row['generic_sentence'].replace(","," <comma>") + ' <end>' for row in ds ]
corpus = np.array(corpus)


Filter:   0%|          | 0/1020868 [00:00<?, ? examples/s]

Create a tokenizer and Detokenizer

In [4]:
tokenizer=TextVectorization( max_tokens=10000, standardize="lower_and_strip_punctuation", encoding="utf-8",) #con il max prende le piu frequenti. ordina i token del vocab dal piu frequente al meno frequente
tokenizer.adapt(corpus)

class TextDetokenizer:
    def __init__(self, vectorize_layer):
        self.vectorize_layer = vectorize_layer
        vocab = self.vectorize_layer.get_vocabulary()
        self.index_to_word = {index: word for index, word in enumerate(vocab)}

    def __detokenize_tokens(self, tokens):
        def check_token(t):
          if t == 3:
            s="<start>"
          elif t == 2:
            s="<end>"
          elif t == 7:
            s="<comma>"
          else:
            s=self.index_to_word.get(t, '[UNK]')
          return s

        return ' '.join([ check_token(token) for token in tokens if token != 0])

    def __call__(self, batch_tokens):
       return [self.__detokenize_tokens(tokens) for tokens in batch_tokens]



detokenizer = TextDetokenizer( tokenizer )
sentences = tokenizer( corpus ).numpy()


Remove from corpus the sentences where any unknow word appears

In [5]:
mask = np.sum( (sentences==1) , axis=1) >= 1
original_data = np.delete( sentences, mask , axis=0)

In [6]:
original_data.shape

(241236, 28)

Shuffle the sentences

In [7]:
def extract_full_data(generator):
    x_list = []
    y_list = []
    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        x_list.append(x_batch)
        y_list.append(y_batch)
    x = np.concatenate(x_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    return x, y

In [8]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, data, batch_size=32, shuffle=True):

        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        data_batch = np.array([self.data[k] for k in indexes])
        #copy of ordered sequences
        result = np.copy(data_batch)
        #shuffle only the relevant positions for each batch
        for i in range(data_batch.shape[0]):
          np.random.shuffle(data_batch[i,1:data_batch[i].argmin() - 1])

        return data_batch , result

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [9]:
train_generator = DataGenerator(original_data[:220000])
test_generator = DataGenerator(original_data[220000:])
x, y = test_generator.__getitem__(1)

In [ ]:
# tokenizer.get_vocabulary()

In [10]:
x = detokenizer(x)
y = detokenizer(y)

for i in range(7):
  print("original: ", y[i])
  print("shuffled: ", x[i])
  print("\n")

original:  <start> wildlife is particularly vulnerable during dry years and in the winter <end>
shuffled:  <start> particularly and vulnerable is in dry the winter during years wildlife <end>


original:  <start> water is essential to the human body because all cell and organ functions depend upon water <end>
shuffled:  <start> all to is organ the water essential functions water human and because depend upon cell body <end>


original:  <start> winter wheat is a good choice to plant in wildlife patches <end>
shuffled:  <start> plant choice wheat to wildlife a in patches good winter is <end>


original:  <start> water can also be lost through stomata <comma> so their opening is regulated by the plant <end>
shuffled:  <start> also so is <comma> opening can regulated the water by their stomata lost be plant through <end>


original:  <start> waves generate interference patterns because they can pass through the two slits simultaneously <end>
shuffled:  <start> simultaneously pass interfer

In [11]:
from difflib import SequenceMatcher

def score(s,p):
  match = SequenceMatcher(None, s, p).find_longest_match()
  #print(match.size)
  return (match.size/max(len(p),len(s)))

In [12]:
original = "at first henry wanted to be friends with the king of france"
generated = "henry wanted to be friends with king of france at the first"

print("your score is ",score(original,generated))

your score is  0.5423728813559322


In [13]:
x_train, y_train = extract_full_data(train_generator)
x_train.shape, y_train.shape

((220000, 28), (220000, 28))

In [14]:
x_test, y_test = extract_full_data(test_generator)
x_test.shape, y_test.shape

((21216, 28), (21216, 28))

In [15]:
x_test[0]

array([   3,   95, 3120,   76,   13,    4,    9,  162,   27,   21,   12,
         25, 2381, 3015,    2,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [16]:
y_test[0]

array([   3,   25, 3120,   95,   21,    4, 2381,   13,   27,  162,   12,
       3015,    9,   76,    2,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [17]:
vocabulary = tokenizer.get_vocabulary()

In [78]:
X_train, Y_train = extract_full_data(train_generator)
X_train.shape, Y_train.shape

((220000, 28), (220000, 28))

In [79]:
from keras.utils import to_categorical, pad_sequences
x_train = pad_sequences(np.array([s[1:] for s in X_train]), maxlen=28, padding='post')
c_train = y_train
y_train = pad_sequences(np.array([s[1:] for s in Y_train]), maxlen=28, padding='post')

In [129]:
import numpy as np
import tensorflow as tf
import keras.backend as k
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.add = tf.keras.layers.Add()
class CrossAttention(BaseAttention):
    def call(self, x, context):
        attn_output, att_scores = self.mha(
            query=x,
            key=context,
            value=context,
            return_attention_scores=True
        )
        self.last_attn_scores = att_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, embedder, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embedding = embedder
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)
    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)
    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[tf.newaxis, :length, :]
        return x
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x
        )
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

class Encoder(tf.keras.layers.Layer):
  def __init__(self, embedder, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = embedder

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # The last attention scores are cached for later plotting
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

class Decoder(tf.keras.layers.Layer):
  def __init__(self, embedder, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(embedder, vocab_size=vocab_size, d_model=d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.embedder = tf.keras.layers.Embedding(input_vocab_size, d_model, mask_zero=True)
    self.encoder = Encoder(self.embedder,num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(self.embedder,num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # All inputs must be passed in the first argument to use '.fit'

    context, x  = inputs
    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Keras mask is dropped, so it doesn't scale with losses or metrics.
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)



# num_layers = 8
num_layers = 8
# d_model = 128
d_model = 128
# dff = 512
dff = 128
# num_heads = 8
num_heads = 8
dropout_rate = 0.2
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=10_000,
    target_vocab_size=10_000,
    dropout_rate=dropout_rate)
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

K_VALUE = 0.97

# Defining a custom loss function that works directly on tokens
def custom_masked_loss(label, pred):

    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_object(label, pred)

    final_array = tf.pow(K_VALUE,tf.cast(tf.range(1,28+1),tf.float32))

    mask = tf.cast(mask, dtype=loss.dtype)
    mask*=final_array

    loss *= mask

    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss

# Defining a custom metric that works directly on tokens
def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

transformer.compile(
    loss=custom_masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

transformer.build(input_shape = [(None, 28), (None, 28)])

transformer.summary()

Model: "transformer_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  1280000   
                                                                 
 encoder_4 (Encoder)         multiple                  5768192   
                                                                 
 decoder_4 (Decoder)         multiple                  9990144   
                                                                 
 dense_92 (Dense)            multiple                  1290000   
                                                                 
Total params: 15768336 (60.15 MB)
Trainable params: 15768336 (60.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
c_train[0], x_train[0]

(array([   3, 3941, 6775,  114,  685,   21, 6429,   16, 1112,  969,   47,
          10,  224,  679,   11, 2756,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]),
 array([   3,  224, 6775,  114,   10,  685,   16, 2756, 1112,   11,  679,
          47, 6429,   21, 3941,  969,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]))

In [126]:
from keras.utils import to_categorical, pad_sequences
c_train = pad_sequences(np.array([s[1:] for s in X_train[:10000]]), maxlen=28, padding='post')
x_train = Y_train[:10000]
y_train = pad_sequences(np.array([s[1:] for s in Y_train[:10000]]), maxlen=28, padding='post')

In [128]:
context_train = []
labels_train = []
input_train = []

for j,x in enumerate(x_train):
  non_null_count = sum(x>0)
  for i in range(non_null_count):
    context_train.append(c_train[j])
    input_train.append(pad_sequences([x[:i+1]], maxlen=28, padding='post')[0])
    labels_train.append(y_train[j])

input_train = np.array(input_train)
labels_train = np.array(labels_train)
context_train = np.array(context_train)

In [121]:
input_train[0],input_train[1],input_train[2]

(array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=int32),
 array([   3, 3941,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0], dtype=int32),
 array([   3, 3941, 6775,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0], dtype=int32))

In [74]:
result = transformer.predict((np.array([c_train[0]]), np.array([x_train[0]])))
tf.argmax(result, axis=2), c_train[0]

1/1 [==============================] - 0s 46ms/step


(<tf.Tensor: shape=(1, 28), dtype=int64, numpy=
 array([[3941, 6775,  679,   11,   10, 1112,   47,   47,   47,   10,   10,
           10,   16, 1112,    2,    2,    2,   11,   11,   11,   11,   11,
           11,   11,   11,   11,   11,   11]])>,
 array([   3, 3941, 6775,  114,  685,   21, 6429,   16, 1112,  969,   47,
          10,  224,  679,   11, 2756,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]))

In [28]:
result.shape

(1, 28, 10000)

In [31]:
y_train[0].shape

(28,)

In [58]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
loss_object(y_train[0],result)

<tf.Tensor: shape=(1, 28), dtype=float32, numpy=
array([[9.292893 , 9.18517  , 9.15374  , 9.255854 , 9.060481 , 8.959764 ,
        9.141303 , 9.333141 , 9.259187 , 9.072947 , 9.064474 , 9.1736145,
        9.151183 , 9.24507  , 9.262337 , 9.192689 , 9.295619 , 9.315495 ,
        9.317844 , 9.304873 , 9.290014 , 9.270765 , 9.259823 , 9.265832 ,
        9.267507 , 9.258655 , 9.23551  , 9.232777 ]], dtype=float32)>

In [59]:
print(custom_masked_loss(y_train[0],result))

tf.Tensor(9.174984, shape=(), dtype=float32)


In [23]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [25]:
import tensorflow as tf

# Check for available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Physical devices cannot be modified after being initialized


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
es = EarlyStopping(monitor='val_masked_accuracy', mode='max', verbose=1, patience=2)

epochs = 20
batch_size= 512

# Training phase
history = transformer.fit(
    (context_train, input_train),
    labels_train,
    epochs=epochs,
    batch_size=batch_size,
    callbacks = [es],
    validation_split = 0.1
)

Epoch 1/20
252/252 [==============================] - 319s 1s/step - loss: 8.5801 - masked_accuracy: 0.0804 - val_loss: 7.6869 - val_masked_accuracy: 0.1300
Epoch 2/20
252/252 [==============================] - 261s 1s/step - loss: 6.9265 - masked_accuracy: 0.0995 - val_loss: 6.6346 - val_masked_accuracy: 0.1061
Epoch 3/20
251/252 [============================>.] - ETA: 0s - loss: 6.3734 - masked_accuracy: 0.1177

In [69]:
result = transformer.predict((np.array([c_train[0]]), np.array([x_train[0]])))
tf.argmax(result, axis=2), x_train[0]

1/1 [==============================] - 0s 80ms/step


(<tf.Tensor: shape=(1, 28), dtype=int64, numpy=
 array([[ 375, 6775,  114,   11,    7,    7,    7,    7,    7,    7,   10,
           10,   16,   16, 6429,    2,   16,   21,   21,   21,   11,   11,
           11,   11,   11,   11,   21,   21]])>,
 array([   3,  224, 6775,  114,   10,  685,   16, 2756, 1112,   11,  679,
          47, 6429,   21, 3941,  969,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0]))

In [ ]:
vocabulary_size = len(vocabulary)
batch_size = 32

In [ ]:
test_generator = ModifiedDataGenerator((x_test,y_test), batch_size=batch_size, vocabulary_size=vocabulary_size)
train_generator = ModifiedDataGenerator((x_train,y_train), batch_size=batch_size, vocabulary_size=vocabulary_size)


Epoch 1/10


# Metrics

Let s be the source string and p your prediction. The quality of the results will be measured according to the following metric:

1.  look for the longest substring w between s and p
2.  compute |w|/max(|s|,|p|)

If the match is exact, the score is 1.

When computing the score, you should NOT consider the start and end tokens.



The longest common substring can be computed with the SequenceMatcher function of difflib, that allows a simple definition of our metric.

Let's do an example.

In [ ]:
original = "at first henry wanted to be friends with the king of france"
generated = "henry wanted to be friends with king of france at the first"

print("your score is ",score(original,generated))

your score is  0.5423728813559322


The score must be computed as an average of at least 3K random examples taken form the test set.

# What to deliver

You are supposed to deliver a single notebook, suitably commented.
The notebook should describe a single model, although you may briefly discuss additional attempts you did.

The notebook should contain a full trace of the training.
Weights should be made available on request.

You must also give a clear assesment of the performance of the model, computed with the metric that has been given to you.

# Good work!